#**Day 1**

**In This day i am going to**

Tasks:

      implement basic RAG:

      Loader → chunker → embeddings → vector store

      Fixed semantic chunking (single strategy)

      Top-k retrieval

Output:
      
      Correct answers for simple questions

      No hallucination (yet)

In [ ]:
!pip install langchain
!pip install langchain-community
!pip install chromadb
!pip install pypdf
!pip install langchain-experimental
!pip install langchain-openai

In [2]:
import pandas as pd
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import CSVLoader,PyPDFLoader
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate
from langchain_core.messages import SystemMessage
from langchain_experimental.text_splitter import SemanticChunker
from transformers import AutoTokenizer,AutoModelForSequenceClassification
from huggingface_hub import login
from sentence_transformers import CrossEncoder
from langchain_community.llms import OpenAI
from langchain_classic.retrievers import ContextualCompressionRetriever
from langchain_classic.retrievers.document_compressors import LLMChainExtractor
from dotenv import load_dotenv
import os


In [ ]:
login()

In [7]:
df = pd.read_csv("/content/employee.csv")

In [8]:
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [ ]:
Embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [5]:
from google.colab import userdata
import os

colab_api_key = userdata.get('KEY').strip()

model = ChatOpenAI(
    model_name="arcee-ai/trinity-large-preview:free",
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=colab_api_key,
)

/tmp/ipython-input-3650021198.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the `langchain-openai package and should be used instead. To use it run `pip install -U `langchain-openai` and import as `from `langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(


In [9]:
pdf = PyPDFLoader("/content/fsi_migration_to_google_cloud.pdf")
pdf = pdf.load()

In [ ]:
csv_file = CSVLoader("/content/employee.csv").load()

In [ ]:
chunker = RecursiveCharacterTextSplitter(separators=['\n\n','\n',". "," ",""])

In [ ]:
chunks = chunker.split_documents(pdf)

In [ ]:
chunks

[Document(metadata={'producer': 'Skia/PDF m104 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'fsi_migration_to_google_cloud', 'source': '/content/fsi_migration_to_google_cloud.pdf', 'total_pages': 21, 'page': 0, 'page_label': '1'}, page_content='FSI MIGRATION TOGOOGLE CLOUD:Key regulatory considerations in the UnitedStates\nPrepared by Global Markets Advisory Group LLCJames Buckley, Louis Pastina, Charles Dolan, and Daniel LabovitzMarch 2022'),
 Document(metadata={'producer': 'Skia/PDF m104 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'fsi_migration_to_google_cloud', 'source': '/content/fsi_migration_to_google_cloud.pdf', 'total_pages': 21, 'page': 1, 'page_label': '2'}, page_content='Table of Contents\nExecutive summary 3\nRegulatory considerations in moving FSI applications to Google Cloud 4\nGoogle’s shared fate model 9\nOperating within Google Cloud 10\nAddressing regulators’ questions and concernsregarding an FSI’s cloud migrati

In [ ]:
vector_store1 = Chroma.from_documents(chunks,Embedding)
vector_store2 = Chroma.from_documents(csv_file,Embedding)

In [ ]:
retrival1 = vector_store1.as_retriever(
    search_type="similarity",
    search_kwargs={"k":5}
)
retrival2 = vector_store2.as_retriever(
    search_type="similarity",
    search_kwargs={"k":5}
)

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a precise assistant. Answer ONLY using the provided context."),
    ("human",
     "Context:\n{context}\n\n"
     "Question:\n{question}")
])


In [ ]:
query = "Explain Identifying areas of risk and implementing strong governance controls"
docs = retrival1.invoke(query)
context = "\n\n".join([d.page_content for d in docs])
rag_chain = template|model
response = rag_chain.invoke({
    "context": context,
    "question": query
})
print(response.content)

Identifying areas of risk and implementing strong governance controls is crucial for Financial Services Institutions (FSIs) when considering migration to cloud services. The process involves a comprehensive risk assessment and the involvement of all relevant stakeholders across the enterprise. Here are the key steps and considerations:

1. Risk Assessment:
   - Evaluate the Cloud Service Provider's (CSP) business experience and qualifications
   - Assess the CSP's financial condition
   - Review legal and regulatory compliance
   - Examine risk management and control processes
   - Assess information security measures
   - Evaluate operational resilience

2. Stakeholder Involvement:
   - CTO/CIO: Technology planning, due diligence, and third-party selection
   - CISO: Information security planning and risk management
   - Finance: Financial planning and due diligence
   - Procurement: Vendor selection and due diligence
   - CCO/Regulatory Compliance: Compliance planning and monitoring


In [ ]:
query = "who has the highest monthly income?"
docs = retrival2.invoke(query)
context = "\n\n".join([d.page_content for d in docs])
rag_chain = template|model
response = rag_chain.invoke({
    "context": context,
    "question": query
})
print(response.content)

The employee with the highest monthly income is the 56-year-old male manager in the Human Resources department, with a monthly income of $19,717.


#**Day 2**

Tasks

Implement:

    Semantic chunking

    Token caps (min/max)

Tune:

    Chunk size

    Overlap

Log failures:

    Missing info

    Mixed topics

Output

    Chosen chunking strategy

    Justification in notes

In [ ]:
chunker3 = SemanticChunker(embeddings = Embedding,breakpoint_threshold_type="percentile",breakpoint_threshold_amount=90)
chunker4 = SemanticChunker(embeddings = Embedding,breakpoint_threshold_type="percentile",breakpoint_threshold_amount=95)

In [ ]:
chunks3 = chunker3.split_documents(pdf)
chunks4 = chunker4.split_documents(pdf)

In [ ]:
vector_store3 = Chroma.from_documents(chunks3,Embedding)
vector_store4 = Chroma.from_documents(chunks4,Embedding)

In [ ]:
retrival3 = vector_store3.as_retriever(
    search_type="similarity",
    search_kwargs = {"k":3}
)
retrival4 = vector_store4.as_retriever(
    search_type="similarity",
    search_kwargs = {"k":3}
)

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a precise assistant. Answer ONLY using the provided context."),
    ("human",
     "Context:\n{context}\n\n"
     "Question:\n{question}")
])


In [ ]:
query = "Effective risk assessment and governance controls"
docs = retrival3.invoke(query)
context = "\n\n".join([d.page_content for d in docs])
rag_chain = template|model
response = rag_chain.invoke({
    "context": context,
    "question": query
})
print(response.content)

Effective risk assessment and governance controls are critical for Financial Services Institutions (FSIs) when considering moving systems, applications, or processes to the cloud. The key elements of a thorough risk assessment process include:

1. Identifying key risks and corresponding mitigating controls
2. Documenting the risk assessment process fully
3. Evaluating the Cloud Service Provider's (CSP) business experience and qualifications
4. Assessing the CSP's financial condition
5. Ensuring legal and regulatory compliance
6. Reviewing risk management and control processes
7. Evaluating information security measures
8. Assessing operational resilience

The risk management process outlined by regulators provides a template for FSIs to follow. Program governance and full vetting by all relevant stakeholders are crucial to the process. A clearly articulated roadmap for implementation is essential to avoid impediments to a successful migration strategy.


In [ ]:
query = "Effective risk assessment and governance controls"
docs = retrival4.invoke(query)
context = "\n\n".join([d.page_content for d in docs])
rag_chain = template|model
response = rag_chain.invoke({
    "context": context,
    "question": query
})
print(response.content)

Effective risk assessment and governance controls are crucial for Financial Services Institutions (FSIs) when considering migration to cloud-based systems. The key elements of a thorough risk assessment process include:

1. Identifying key risks and corresponding mitigating controls
2. Documenting the risk assessment process and findings
3. Evaluating the Cloud Service Provider's (CSP) business experience and qualifications
4. Assessing the CSP's financial condition
5. Reviewing the CSP's legal and regulatory compliance
6. Examining the CSP's risk management and control processes
7. Evaluating the CSP's information security measures
8. Assessing the CSP's operational resilience

To ensure effective governance controls, FSIs should:

1. Follow the risk management process outlined by regulators as a template
2. Implement program governance and full vetting by all relevant stakeholders
3. Adopt a clearly articulated roadmap for implementation
4. Assess issues and identify responsive mater

##For the query "Effective risk assessment and governance controls", both 90 and 95 percentile semantic chunking configurations retrieved relevant content.
##The 90 percentile configuration produced a more faithful, tightly scoped answer aligned with the source text, while 95 introduced additional abstraction.
##Given the regulatory nature of the content, 90 percentile was selected for higher faithfulness and lower risk of over-summarization.


#**Day 3**

Query Enhancement

Tasks

    Query transformations

    HyDE

Test:

    Raw query vs enhanced query

    Output

    Improved recall

    Notes on failure cases

In [ ]:
chunker = SemanticChunker(embeddings = Embedding,breakpoint_threshold_type="percentile",breakpoint_threshold_amount=90)
chunks = chunker.split_documents(pdf)
vector_store = Chroma.from_documents(chunks,Embedding)

In [ ]:
def query_rewrite(query):
  template = PromptTemplate(
      input_variables=["query"],
      template=(
          "You are an expert in information retrieval.\n"
          "Rewrite the following user query to be more specific, detailed, "
          "and aligned with terminology likely used in technical and regulatory documents.\n"
          "Do NOT answer the query.\n"
          "Return ONLY the rewritten query.\n\n"
          "Original query: {query}"
          )
  )
  chain = template|model
  return chain.invoke({"query":query})

In [ ]:
rewritten = query_rewrite("Effective risk assessment and governance controls")
print(rewritten)

content='"Comprehensive evaluation of risk assessment methodologies and implementation of robust governance controls in compliance with industry standards and regulatory frameworks."' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 62, 'total_tokens': 85, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'cost': 0, 'is_byok': False, 'cost_details': {'upstream_inference_cost': 0, 'upstream_inference_prompt_cost': 0, 'upstream_inference_completions_cost': 0}}, 'model_name': 'arcee-ai/trinity-large-preview:free', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019c4cc4-a0fa-7443-a458-f19cd840c6bf-0' tool_calls=[] invalid_tool_calls=[]


In [ ]:
retrival = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs = {"k":5}
)

In [ ]:
docs = retrival.invoke(rewritten.content)
context = "\n\n".join([d.page_content for d in docs])
rag_chain = template|model
response = rag_chain.invoke({
    "context": context,
    "question": rewritten.content
})
print(response.content)

To ensure compliance with industry standards and regulatory frameworks, FSIs must conduct a comprehensive evaluation of risk assessment methodologies and implement robust governance controls. This involves assessing the risks associated with outsourcing critical business, technology, and regulatory compliance functions to third-party vendors, such as cloud service providers (CSPs). The FSI should have a clear understanding of the controls that the CSP is responsible for managing and those that the FSI is responsible for configuring and managing. Additionally, FSIs must establish effective risk assessment and governance controls, including protocols for incident management and reporting, to address potential system-related issues that may impact the performance or compliance of cloud-hosted systems. Regulators expect FSIs to maintain a robust vendor management program, conduct testing of CSPs, and provide ongoing oversight to ensure compliance with regulatory obligations.


In [ ]:
def Hyde(query):
  template = PromptTemplate(
      input_variables=["query"],
      template=(
          "You are an expert in information retrieval.\n"
          "answer the following question using a hypothetical answer for it"
          "and aligned with terminology likely used in technical and regulatory documents.\n"
          "Return ONLY the Hypothetical answer.\n\n"
          "Original query: {query}"
          )
      )
  chain = template|model
  return chain.invoke({"query":query})

In [ ]:
Hypothetical_Answer = Hyde("Effective risk assessment and governance controls").content

In [ ]:
docs = retrival.invoke(Hypothetical_Answer)
context = "\n\n".join([d.page_content for d in docs])
rag_chain = template|model
response = rag_chain.invoke({
    "context": context,
    "question": Hypothetical_Answer
})
print(response.content)

Effective risk assessment and governance controls require a structured framework that integrates risk identification, evaluation, and mitigation strategies within the organization’s operational and regulatory context. This involves the establishment of clear governance structures, including defined roles and responsibilities for risk oversight, and the implementation of robust policies and procedures to ensure compliance with applicable laws, regulations, and industry standards. Key components include continuous monitoring of risk exposure, regular audits, and the use of advanced analytics to predict and respond to emerging risks. Additionally, governance controls should incorporate mechanisms for reporting, escalation, and resolution of identified risks, ensuring alignment with organizational objectives and stakeholder expectations.


##From Day 3 I conclude that Query Rewriting improved signficantly the answers of the model to the Retrived Docs

#**Day 4**

Tasks

Implement:

    Reranker (preferred)


Compare:

    Before vs after

Output:

    Clear retrieval improvement

In [ ]:
docs = retrival.invoke(rewritten.content)

In [ ]:
Reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L12-v2')

In [ ]:
def rerank(query,docs):
  pairs = [(query,d.page_content) for d in docs]
  scores = Reranker.predict(pairs)
  ranked = sorted(
      zip(docs,scores),
      key=lambda x:x[1],
      reverse=True
  )
  return [doc for doc, _ in ranked]

In [ ]:
top_docs = rerank(rewritten.content,docs)

In [ ]:
top_docs

[Document(metadata={'creator': 'PyPDF', 'page_label': '6', 'creationdate': '', 'page': 5, 'total_pages': 21, 'source': '/content/fsi_migration_to_google_cloud.pdf', 'producer': 'Skia/PDF m104 Google Docs Renderer', 'title': 'fsi_migration_to_google_cloud'}, page_content='Depending on thecircumstances, the FSI may need to communicate any system-related issues impactingthe performance or compliance of its cloud-hosted systems to its regulators within adeﬁned (and in some cases very short) timeframe. The protocols established betweenthe FSI and the CSP for incident management and reporting should address thispotentiality and provide for prompt notiﬁcations between the parties and a documentedprocess for investigating, determining the root cause of the issue, timely reporting, andresolution. E\x00ective risk assessment and governance controls\nThe regulators in the banking, securities and commodities sectors have long recognizedthat FSIs can outsource key business, technology and regulator

In [ ]:
context = "\n\n".join([d.page_content for d in top_docs])
rag_chain = template|model
response = rag_chain.invoke({
    "context": context,
    "question": rewritten.content
})
print(response.content)

To conduct comprehensive risk assessments and implement governance controls in compliance with relevant regulatory standards and industry guidelines, FSIs should follow these methodologies, frameworks, and best practices:

1. Risk Assessment:
   - Conduct thorough due diligence on third-party vendors, including CSPs
   - Perform ongoing monitoring of vendor performance and compliance
   - Assess risks in critical areas such as cybersecurity, data governance, business continuity, and recordkeeping
   - Ensure risk assessment is commensurate with the level of risk associated with the activity

2. Governance Controls:
   - Establish clear protocols for incident management and reporting between FSI and CSP
   - Implement robust vendor management programs
   - Define roles and responsibilities for both FSI and CSP in managing controls
   - Conduct regular testing of CSPs as part of due diligence obligations

3. Regulatory Compliance:
   - Familiarize with relevant regulations and guidance f

#**Day 5**

Context Control

Tasks

Add:

    Contextual compression


Ensure:

    No irrelevant paragraphs


Output

    Cleaner context passed to LLM

    Fewer hallucinations

In [ ]:
compressor = LLMChainExtractor.from_llm(model)
retrive = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retrival
)

In [ ]:
docs = retrive.invoke(rewritten.content)

In [ ]:
top_docs = rerank(rewritten.content,docs)

In [ ]:
context = "\n\n".join([d.page_content for d in top_docs])
rag_chain = template|model
response = rag_chain.invoke({
    "context": context,
    "question": rewritten.content
})
print(response.content)

To address this, FSIs must implement a comprehensive risk assessment methodology that is commensurate with the level of risk associated with the outsourced activity. This involves a thorough evaluation of potential vendors, their controls, and their ability to meet regulatory obligations. The governance process should include clear delineation of responsibilities between the FSI and the CSP, with a focus on critical areas such as cybersecurity, data governance, business continuity, and recordkeeping. Regular testing and monitoring of CSPs should be conducted as part of the due diligence obligations, with regulators likely to review test plans, performance, and follow-up results. A robust vendor management program is essential to ensure ongoing oversight and supervision of CSP performance. This program should cover all stages from planning and due diligence to ongoing monitoring and notification of changes or incidents. By adhering to these practices and following industry standards and

#**Day 6**

asks

Implement refusal logic:

    Based on context recall / relevance

Add explicit:

    “Insufficient information” response

Output

    System refuses when it should

    Documented logic

In [ ]:
docs = retrival.invoke(rewritten.content)

In [ ]:
pairs = [(rewritten.content,d.page_content) for d in docs]

In [ ]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L12-v2')

In [ ]:
scores = cross_encoder.predict(pairs)

In [ ]:
scores

array([-0.22764578, -6.627781  , -0.0486474 , -7.2313776 , -1.4439518 ],
      dtype=float32)

In [ ]:
T = -1.0
filtered_docs = [
    doc for doc, score in zip(docs, scores) if score >= T
]

In [ ]:
if(len(filtered_docs)==0):
  print("The documents do not contain sufficient information to answer this question.")

#**Day 7**
**Putting all together**

In [10]:
chunker = SemanticChunker(embeddings = Embedding,breakpoint_threshold_type="percentile",breakpoint_threshold_amount=90)
chunks = chunker.split_documents(pdf)
vector_store = Chroma.from_documents(chunks,Embedding)

In [11]:
def Hyde(query):
  template = PromptTemplate(
      input_variables=["query"],
      template=(
          "You are an expert in information retrieval.\n"
          "answer the following question using a hypothetical answer for it"
          "and aligned with terminology likely used in technical and regulatory documents.\n"
          "Return ONLY the Hypothetical answer.\n\n"
          "Original query: {query}"
          )
      )
  chain = template|model
  return chain.invoke({"query":query})

In [12]:
Hypothetical_Answer = Hyde("Effective risk assessment and governance controls").content

In [ ]:
Reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L12-v2')

In [17]:
retrival = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs = {"k":5}
)

In [14]:
def rerank(query,docs):
  pairs = [(query,d.page_content) for d in docs]
  scores = Reranker.predict(pairs)
  ranked = sorted(
      zip(docs,scores),
      key=lambda x:x[1],
      reverse=True
  )
  return [doc for doc, _ in ranked]

In [18]:
docs = retrival.invoke(Hypothetical_Answer)

In [19]:
top_docs = rerank(Hypothetical_Answer,docs)

In [ ]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L12-v2')

In [22]:
pairs = [(Hypothetical_Answer,d.page_content) for d in top_docs]

In [23]:
scores = cross_encoder.predict(pairs)

In [24]:
scores

array([ 0.55007035, -0.32286817, -0.6016444 , -0.8103631 , -1.6661713 ],
      dtype=float32)

In [25]:
T = 0.3
filtered_docs = [
    doc for doc, score in zip(docs, scores) if score >= T
]

In [26]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a precise assistant. Answer ONLY using the provided context."),
    ("human",
     "Context:\n{context}\n\n"
     "Question:\n{question}")
])


In [27]:
if(len(filtered_docs)==0):
  print("The documents do not contain sufficient information to answer this question.")
else:
  context = "\n\n".join([d.page_content for d in filtered_docs])
  rag_chain = template|model
  response = rag_chain.invoke({
      "context": context,
      "question": Hypothetical_Answer
  })
  print(response.content)

Effective risk assessment and governance controls are crucial for Financial Services Institutions (FSIs) when utilizing cloud services. These controls involve a systematic process of identifying, evaluating, and mitigating potential risks to ensure organizational resilience and compliance with regulatory requirements. Key components include establishing a robust governance framework, implementing risk management policies, conducting regular risk assessments, and maintaining oversight through internal audits and reporting mechanisms. The FSI must have a clear understanding of the controls that the Cloud Service Provider (CSP) is responsible for managing and those controls that the FSI is responsible for configuring and managing. This understanding is essential for maintaining compliance and operational effectiveness in critical areas such as cybersecurity, data governance, business continuity, and recordkeeping. The level of due diligence and oversight should be commensurate with the ri